# Topic Extraction in Newsfeeds
* Autor: Prof. Dr. Johannes Maucher
* Datum: 17.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.html)

#Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __RSS Feeds:__ Struktur von RSS Feeds analysieren und parsen mit dem _Universal Feed Parser_. 
* __Dokument Analyse:__ Die Häufigkeit aller Worte in einem Dokument (Inhalt des RSS Feeds) zählen und in einem Array verwalten. 
* __Merkmalsextraktion:__ Bestimmung von Merkmalen (hier auch: __Topics__) (Allgemein spricht man von Merkmalen. Im Fall, dass die NNMF auf Dokumente angewandt wird, werden die Merkmale auch mit __Topics__ oder __Themen__ bezeichnet) mit der \emph{Non Negative Matrix Factorization}.
* __Zuordnung__: Wie setzen sich die Topics aus den Wörtern zusammen? Wie stark sind die gefundenen Topics in den Artikeln vertreten?
* __Dokument Clustering:__ Mit der NNMF kann auch ein Clustering realisiert werden. Jeder Topic repräsentiert ein Cluster. Jedes Dokument wird dem Cluster zugeordnet, dessen Topic am stärksten in ihm vertreten ist. 

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Stellen Sie sich vor Sie möchten in eine eigene Webseite die RSS Feeds einer Menge von Nachrichtenservern einbinden. Da die unterschiedlichen Server wahrscheinlich Artikel zu den gleichen Themen anbieten, werden die Inhalte einiger Artikel ähnlich sein. Mit der __Nicht Negativen Matrixfaktorisierung (NNMF)__ kann für eine große Menge von Dokumenten eine Menge von Themen (Topics) ermittelt werden, auf die sich die Dokumente beziehen. Damit ist es u.a. möglich
* die Dokumente thematisch zu ordnen
* zu jedem Thema nur ein Dokument anzuzeigen

### Ähnlichkeiten bestimmen und relevante Merkmale extrahieren

Eine Sammlung von Dokumenten - in diesem Versuch die Menge aller Nachrichten der angegebenen Feeds - kann in einer Artikel/Wort-Matrix repräsentiert werden. Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile $i$, Spalte $j$ beschreibt wie häufig das Wort in Spalte $j$ im zur Zeile $i$ gehörenden Dokument vorkommt.

Unter der Annahme, dass Artikel umso ähnlicher sind, je mehr Worte in diesen gemeinsam vorkommen, kann auf der Grundlage dieser Matrix die Ähnlichkeit zwischen den Artikeln berechnet werden. Hierzu könnte die Matrix z.B. einfach einem _Hierarchischen Clustering_ übergeben werden. Das hierarchische Clustering weist jedoch im Fall einer großen Menge von zu vergleichenden Objekten zwei wesentliche Nachteile auf: Erstens ist die wiederholte Berechnung der Distanzen zwischen allen Artikeln/Clustern extrem rechenaufwendig, zweitens ist die Darstellung einer großen Anzahl von Objekten im Dendrogramm nicht mehr übersichtlich. 

Für das Auffinden von Assoziationen zwischen Dokumenten hat sich in den letzten Jahren die Methode der __Nicht-Negativen Matrix Faktorisierung (NNMF)__ etabliert. Mit dieser Methode kann eine Menge von wesentlichen Merkmalen berechnet werden, anhand derer sich die Dokumente clustern lassen, d.h. Dokumente des gleichen Clusters repräsentieren das gleiche Merkmal (Thema). Ein solches Merkmal wird durch eine Menge von Worten beschrieben, z.B. $\{$ _Paris, terror, IS_ $\}$  oder $\{$_refugee, syria, border_ $\}$. Neben der Merkmalsextraktion stellt die relativ geringe Komplexität einen weiteren Vorteil der NNMF dar. Durch die Darstellung der Artikel/Wort-Matrix als Produkt von 2 Faktormatrizen müssen deutlich weniger Einträge gespeichert werden.

### Nicht Negative Matrixfaktorisierung: Die Idee

Die Artikel/Wort-Matrix wird im Folgenden mit $A$ bezeichnet. Sie besitzt $r$ Zeilen und $c$ Spalten, wobei $r$ die Anzahl der Artikel und $c$ die Anzahl der relevanten Worte in der Menge aller Artikel ist. Durch Multiplikation der Matrix $A$ mit dem Vektor $v$ (_wordvec_: Vektor der alle relevanten Worte enthält) werden die Worte den Artikeln $a$ (_articletitles_: Vektor der alle Artikeltitel enthält) zugeordnet:

$$
a=A*v.
$$

Die Idee der NNMF besteht darin die Matrix $A$ als Produkt zweier Matrizen $W$ und $H$ darzustellen,

$$
A=W*H
$$

wobei alle Elemente in $W$ und $H$ größer oder gleich Null sein müssen. Die Matrixmultiplikation erfordert, dass die Anzahl der Zeilen $m$ in $H$ gleich der Anzahl der Spalten in $W$ sein muss. 
Durch die Faktorisierung der Matrix $A$ wird die Zuordnung der Wörter des Wortvektors $v$  zu den Artikeln des Vektors $a$ in zwei Stufen zerlegt. 

$$
f = H*v
$$
$$
a = W*f 
$$

In der ersten Stufe werden durch die Multiplikation von $v$ mit der Matrix $H$ die Wörter einem sogenannten Merkmalsvektor $f$ mit $m$ Elementen zugewiesen. In der zweiten Stufe werden durch die Multiplikation des Merkmalsvektor $f$ mit der Matrix $W$ die einzelnen Merkmale den Artikeln in $a$ zugeordnet. Die Matrix $H$ definiert also aus welchen Wörtern die Merkmale gebildet werden. Sie wird deshalb __Merkmalsmatrix__ genannt. Die Matrix $W$ hingegen beschreibt mit welchem Gewicht die einzelnen Merkmale in den verschiedenen Artikeln auftreten. Sie wird deshalb __Gewichtungsmatrix__ genannt.

Daraus folgt: Wenn eine Faktorisierung der Matrix $A$ gefunden wird, dann werden damit auch relevante Merkmale, also die Themen, definiert, hinsichtlich derer die Artikel effizient kategorisiert werden. Durch die Matrixfaktorisierung wird eine __Merkmalsextraktion__ realisiert. 

### Berechnung der Matrixfaktoren

Für die Berechnung der Faktoren wurde in [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) eine iterative Methode vorgestellt, die derzeit wohl am häufigsten angewandt wird und auch in dieser Übung implementiert werden soll. Der Algorithmus besteht aus folgenden Schritten:
* Gebe die zu faktorisierende Matrix $A$ ein. $r$ sei die Anzahl der Zeilen und $c$ die Anzahl der Spalten von $A$.
* Wähle die Anzahl $m$ der Merkmale, mit $m<c$. _Tipp:_ Für $m$ sollte zunächst ein Wert im Bereich $15$ bis $30$ gewählt werden.
* Lege eine $m \times c$ Matrix $H$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Lege eine $r \times m$ Matrix $W$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Wiederhole bis maximale Anzahl der Iteration erreicht oder Kosten $k$ unter vordefinierter Schwelle:

	* Berechne aktuelles Produkt $B=W*H$ und bereche die Kostenfunktion 
		$$
			k=\left\| A - B \right\|^2 = \sum\limits_{i,j} \left(A_{i,j} - B_{i,j}\right)^2
		$$ 
	* Anpassung der Matrix $H$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		H_{i,j} := H_{i,j} \frac{(W^T*A)_{i,j}}{(W^T*W*H)_{i,j}}
		$$
        
	* __Nach__ der Anpassung der Matrix $H$: Anpassung der Matrix $W$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		W_{l,i} := W_{l,i} \frac{(A*H^T)_{l,i}}{(W*H*H^T)_{l,i}}
		$$

In [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) ist bewiesen, dass durch die o.g. Anpassungsroutinen die Kosten $k$ monoton abnehmen und in einem Minimum konvergieren. Der Algorithmus ist jedoch nicht optimal weil das gefundene Minimum ein lokales Minimum sein kann.

 ## Vor dem Versuch zu klärende Fragen
 
 * Was versteht man unter Artikel/Wort-Matrix? Wie wird diese im aktuellen Versuch gebildet?

In [ ]:
#Your markdown

* Wie multipliziert man die Matrix
    $$
    A= \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
    $$
    mit dem Vektor  
    $$
    v=\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
    $$
    

In [ ]:
#Your markdown

* Was versteht man im Kontext der NNMF unter
    * Merkmalsmatrix
    * Gewichtsmatrix

In [ ]:
#Your markdown

* Wie definieren die Zeilen der Merkmalsmatrix die einzelnen Merkmale (Topics)?
* Was definieren die Zeilen der Gewichtungsmatrix?

In [ ]:
#Your markdown

* Wie werden in Numpy zwei Arrays (Typ numpy.array) 
	* im Sinne der Matrixmultiplikation miteinander multipliziert?
	* elementweise multipliziert?
* Wie wird die Transponierte eines Numpy-Arrays berechnet?

In [ ]:
#Your code

# Versuchsdurchführung
Die in diesem Versuch einzubindenden Feeds sind in der unten stehenden Liste _feedlist_ definiert. Die aus dem vorigen Vesuch bereits bekannte Funktion _stripHTML()_ ist ebenfalls gegeben:

In [1]:
import feedparser

feedlist=['http://feeds.reuters.com/reuters/topNews',
          'http://feeds.reuters.com/reuters/businessNews',
          'http://feeds.reuters.com/reuters/worldNews',
          'http://feeds2.feedburner.com/time/world',
          'http://feeds2.feedburner.com/time/business',
          'http://feeds2.feedburner.com/time/politics',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml',
          'http://feeds.nytimes.com/nyt/rss/Business',
          'http://www.nytimes.com/services/xml/rss/nyt/World.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/Economy.xml'
          ]

In [2]:
def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p

## Anlegen der Artikel/Wort-Matrix

### Die Funktion _getarticlewords()_
Schreiben Sie eine Funktion _getarticlewords()_, die folgende Elemente zurückgibt:

* _allwords:_ ist ein Dictionary dessen Keys die Worte aller gesammelten Artikel sind. Der zu jedem Key gehörende Wert ist die Anzahl, wie oft das Wort insgesamt vorkommt.
* _articlewords:_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Listenelement ist ein Dictionary, welches die Worte des jeweiligen Artikels als Key enthält und als Wert die Worthäufigkeit.
* _articletitles_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Element ist der Artikeltitel als String.

In [37]:
import nltk

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

In [38]:
def getwordsTFStop(docText):    
    sentence = nltk.regexp_tokenize(docText, r"[\wäöüÄÖÜß]+")
    
    tokens = [x.lower() for x in sentence if x.lower() not in stops]
    
    wordDict = { x: tokens.count(x) for x in tokens }
    
    return wordDict

Für das Parsing der Feeds soll wieder das Modul _feedparser_ eingesetzt werden. Die zu einer Nachricht gehörenden Wörter sollen die Wörter des Elements _title_ und die Wörter des Elements _description_ sein (siehe voriger Versuch). Allerdings sollen hier nicht alle Wörter eingebunden werden, sondern wie im vorigen Versuch eine Methode _getwords()_ implementiert werden, welche nur die _relevanten_ Wörter zurückgibt. Die Frage welche Wörter relevant sind ist nicht eindeutig beantwortbar. Sie können sich hierzu eigene Antworten einfallen lassen. Auf jeden Fall sollten aber die Stopwörter ignoriert werden. Hierzu kann z.B. die Stopwortliste von NLTK angewandt werden.

In [106]:
def get_article_words():
    comb_text = ''
    single_texts = []
    article_titles = []
    
    for feed in feedlist:
        f=feedparser.parse(feed)
        for e in f.entries:
            if e.get('description', 0):
                article_titles.append(e.title)
                single_texts.append(stripHTML(e.title+'\n'+e.description))
                comb_text += stripHTML(e.title+'\n'+e.description + '\n')
                
    all_words = getwordsTFStop(comb_text)
    remove_list = get_remove_list(all_words, single_texts)
    article_words = [({k: v for k, v in getwordsTFStop(text).items() if k not in remove_list}, index)\
                     for index, text in enumerate(single_texts)]
    
    curr_minus = 0
    for word_dict, index in article_words:
        if len(word_dict) == 0:
            del article_titles[index + curr_minus]
            curr_minus -= 1
            
    article_words = [w_dict for w_dict, index in article_words if len(w_dict) > 0]
        
    return (all_words, article_words, article_titles)

Nachdem alle relevanten Wörter aller Nachrichten gesammelt sind, sollte eine weitere Bereinigung stattfinden, die 

* alle Wörter, die weniger als 4 mal vorkommen
* alle Wörter, die in mehr als 30% aller Dokumente vorkommen

entfernt. 

Durch dieses Herausfiltern nicht relevanter Wörter kann es vorkommen, dass einzelne Artikel keine relevanten Wörter mehr enthalten. Diese Artikel sollen dann ganz ignoriert werden. D.h. unter anderem, dass diese Artikel auch nicht in _articlewords_ und _articletitles_ erscheinen.

In [107]:
def get_remove_list(all_words, single_texts):
    MIN_OCCURENCE = 4
    MAX_OCCURENCE = 0.3
    
    remove_list = []
    
    for key, value in all_words.items():
        key_occurrence = 0
        for text in single_texts:
            if key in text:
                key_occurrence += 1
        
        if value < 4 or key_occurrence > (len(single_texts) / 3):
            remove_list.append(key)
            del all_words[key]
    
    return remove_list

In [108]:
all_words, article_words, article_titles = get_article_words()

In [112]:
article_titles[:10]

[u'Trump says he will back away from business to focus on White House',
 u'Oil price rally likely short-lived as OPEC deal not enough to reduce glut',
 u'Soccer plane crashed without fuel, officials investigating',
 u'Jobless claims rise to five-month high',
 u"Reunited in death: Fidel Castro's remains rest at Che Guevara mausoleum",
 u"Death toll rises to seven in Tennessee's Great Smoky Mountains fires",
 u'Putin curbs anti-Western rhetoric, says wants to get on with Trump',
 u'Ukraine says missile tests will avoid Crimea, mollifying Russia',
 u"Citic-Carlyle near deal to buy McDonald's China assets: source",
 u'Separation anxiety: Trump\u2019s management style poses challenges in Oval Office']

### Die Funktion _makematrix()_
Schreiben Sie eine Funktion _makematrix()_, die aus dem Dictionary _allwords_ und der Liste _articlewords_ (vorige Aufgabe) die Artikel-/Wort-Matrix generiert. Die Einträge in der Matrix sollen die Häufigkeiten der Wörter im jeweiligen Dokument sein (term frequency tf). Die Artikel-/Wort-Matrix soll als 2-dimensionales Numpy Array angelegt werden.

In [119]:
import numpy as np

In [120]:
def make_matrix(all_words, article_words):
    return np.array([[article.get(word, 0) for word in all_words.keys()] for article in article_words])

In [123]:
make_matrix(all_words, article_words)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Die Nicht Negative Matrix Faktorisierung
Die Implementierung der NNMF ist entsprechend der Beschreibung im Theoriekapitel durchzuführen.

* Implementieren Sie die Funktion _cost(A,B)_. Dieser Funktion werden zwei Numpy-Matrizen $A$ und $B$ übergeben. Zurück geliefert werden die nach oben angegebener Formel berechneten Kosten $k$. Diese Funktion wird von der im folgenden beschriebenen Funktion _nnmf(A,m,it)_ benutzt.
* Implementieren Sie die Funktion __nnmf(A,m,it)__. In dieser Funktion soll der oben beschriebene Algorithmus für die Nicht-negative Matrix Faktorisierung ausgeführt werden. Der Funktion wird die zu faktorisierende Matrix $A$, die Anzahl der Merkmale $m$ und die Anzahl der Iterationen $it$ übergeben. Die Funktion gibt die gefundenen Faktoren $W$ und $H$ zurück. In jeder Iteration sollen mit der Funktion __cost(A,B)__ die Kosten berechnet werden. Sobald die Kostenabnahme pro 10 Iterationen kleiner als $2$ ist oder eine maximale Anzahl von Iterationen ($maxIt=200$) erreicht ist, soll der Algorithmus mit der Rückgabe der Faktoren $W$ und $H$ terminieren.     


Tipp für die Implementierung elementweiser Operationen von Matrizen: Für elementweise Operationen müssen in Python/Numpy nicht alle Elemente über Schleifen explizit berechnet werden. Eine elementweise Anpassung aller Matrixelemente kann kompakt programmiert werden indem die beteiligten Matrizen für diese Operationen als Arrays implementiert werden. Sollen z.B. die beiden gleich großen Numpy Arrays $U$ und $V$ elementweise multipliziert werden, dann wäre der entsprechende Programmcode einfach _U*V_.  

In [ ]:
#Your code

## Anzeige der Merkmale und der Gewichte

Im vorigen Abschnitt wurde die Merkmalsmatrix $H$ und die Gewichtsmatrix $W$ berechnet. Diese Matrizen können natürlich am Bildschirm ausgegeben werden, was jedoch nicht besonders informativ ist. Aus den Matrizen können jedoch die Antworten für die folgenden interessanten Fragen berechnet werden:

* In welchen Artikeln sind welche Merkmale stark vertreten?
* Wie lassen sich die insgesamt $m$ Merkmale beschreiben, so dass aus dieser Merkmalsbeschreibung klar wird, welches Thema den Artikeln, in denen das Merkmal stark vertreten ist, behandelt wird? 
 
Die Antwort auf die erste Frage ergibt sich aus der Gewichtsmatrix $W$. Für die Beantwortung der zweiten Frage wird die Merkmalsmatrix $H$ herangezogen.



### Beschreibung der Merkmale

Die Merkmalsmatrix $H$ beschreibt, wie stark die Worte aus _wordvec_ in jedem Merkmal enthalten sind. Jede Zeile von $H$ gehört zu einem Merkmal, jede Spalte von $H$ gehört zu einem Wort in _wordvec_.

Es bietet sich an jedes Merkmal einfach durch die $N=6$ Wörter aus _wordvec_ zu beschreiben, welche am stärksten in diesem Merkmal enthalten sind. Hierzu muss für jedes Merkmal die entsprechende Zeile in $H$ nach den $N=6$ größten Werten durchsucht bzw. geordnet werden. Die entsprechenden Spalten dieser Matrixelemente verweisen dann auf die $N=6$ wichtigsten Worte des Merkmals.

Tipp für die Implementierung: Legen Sie für jedes Merkmal $i$ eine Liste an. Die Listenlänge ist durch die Anzahl der Worte in _wordvec_ (d.h. die Anzahl der Spalten in $H$) gegeben. Jedes Listenelement $j$ enthält selbst wieder 2 Elemente: An erster Stelle den entsprechenden Wert $H_{i,j}$ der Merkmalsmatrix, an der zweiten Stelle das $j.$-te Wort in _wordvec_. Nachdem die Liste angelegt ist, kann sie mit _listname.sort()_ in aufsteigender Reihenfolge sortiert werden. Die abnehmende Sortierung erhält man mit _listname.sort().reverse()_. Danach geben die $N=6$ ersten Listenelemente die für das Merkmal $i$ wichtigsten Worte an.

   
### Präsenz der Merkmale in den Artikeln

Die Gewichtsmatrix $W$ beschreibt, wie stark die $m$ Merkmale in den Artikeln aus _articletitles_ enthalten sind. Jede Zeile von $W$ gehört zu einem Artikel, jede Spalte von $W$ gehört zu einem Merkmal.
Die Berechnung der $M=2$ gewichtigsten Merkmale für jeden Artikel in _articletitles_ kann analog zu der oben beschriebenen Berechnung der $N=6$ wichtigsten Worte eines Merkmals berechnet werden.


### Implementierung

Implementieren Sie eine Funktion _showfeatures(w,h,titles,wordvec)_, welche wie oben beschrieben für jeden Artikel die $M=2$ wichtigsten Merkmale am Bildschirm ausgibt. Dabei soll jedes Merkmal durch die 6 wichtigsten Wörter dieses Merkmals angegeben werden. Siehe Beispielausgabe unten.  

Übergabeparameter der Funktion sind die Merkmalsmatrix $H$, die Gewichtungsmatrix $W$, die Liste aller Artikeltitel _articletitles_ und die Liste aller Worte _wordvec_.


Beispiel fuer Ausgabe:

[(13.54131155883748, 13, u'Putin vows payback after confirmation of Egypt plane bomb'),

(2.2466669548146254, 9, u'Putin vows payback after confirmation of Egypt plane bomb')]

----- ['plane', 'egypt', 'russia', 'month', 'killing', 'putin']

----- ['airport', 'russian', 'crash', 'egypt', 'security', 'officials']

Die Ausgabe ist wie folgt zu interpretieren:
* Für den Artikel _Putin vows payback after confirmation of Egypt plane bomb_ ist 
    * das wichtigste Merkmal durch die 6 Wörter _plane_, _egypt_, _russia_, _month_, _killing_, _putin_ definiert. Das Gewicht dieses Merkmals im Artikel ist 13.54
    * das zweitwichtigste Merkmal durch die 6 Wörter _airport_, _russian_, _crash_, _egypt_, _security_, _officials_ definiert. Das Gewicht dieses Merkmals im Artikel ist 2.24

In [ ]:
#Your code

## Aufgaben

1. Analysieren Sie die berechneten Topics indem Sie sich überlegen ob die gefundenen 6 Wörter pro Topic wirklich Themen beschreiben.
2. Verändern Sie die Parameter der NNMF (Anzahl der Topics $m$, Anzahl der Iterationen). Bei welcher Einstellung der Parameter erhalten Sie das für sie sinnvollste Resultat (sinnvolle Topics)?
3. Wie kann die _getwords()_ Methode verbessert werden, so dass noch bedeutsamere Topics gefunden werden? 

In [7]:
#Your markdown